In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: Machine Learning

In [3]:
#http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Machine+Learning&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Machine+Learning

In [4]:
paper_link_list = []
#249
for i in range(249):
    before_url = "http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Machine+Learning&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Machine+Learning"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

PSO 알고리즘을 이용한 퍼지 Extreme Learning Machine 최적화
Extreme Learning Machine 기반 퍼지 패턴 분류기 설계
Lifelong Machine Learning 기반 스팸 메시지 필터링 방법
행정-정책 의사결정에서 머신러닝(machine learning) 방법론 도입의 정책적 함의: 기계의 한계와 증거기반 의사결정(evidence-based decision-making)
Machine Learning based Prediction of The Value of Buildings
Fast Face Gender Recognition by Using Local Ternary Pattern and Extreme Learning Machine
Extreme Learning Machine을 이용한 자기부상 물류이송시스템 모델링
Extreme Learning Machine Ensemble Using Bagging for Facial Expression Recognition
Machine Learning을 이용한 무기 체계(or 구성품) 고장 유형 식별
A Machine-Learning Based Approach for Extracting Logical Structure of a Styled Document
10
Extreme Learning Machine을 이용한 자기부상 물류이송시스템 모델링
A novel visual tracking system with adaptive incremental extreme learning machine
Comparing Machine Learning Classifiers for Movie WOM Opinion Mining
Prediction & Assessment of Change Prone Classes Using Statistical & Machine Learning Techniques
XSSClassifier: An Efficient XSS Attack Detection Approach Bas

다중 머신러닝 알고리즘을 이용한 악성 URL 예측 시스템 설계 및 구현
잠재계층성장분석과 머신러닝에 의한  초등학교 저학년 학업수행능력의 종단 유형 예측
UX 디자인 과정에서의 머신러닝 활용 방법
LBSN(Location-Based Social Network) 데이터와 머신러닝 기법을 이용한 토지이용 분류
비과 영역에서 머신러닝의 적용
인적자원의 부정채용을 감시하기 위한 머신러닝의 활용
정적 변형률 데이터 기반 머신러닝에 의한 무도상 철도 판형교의 손상 탐지
융자성 기금관리를 위한 블록체인, 머신러닝 설계 연구
쉴드 TBM 기계 데이터 및 머신러닝 기법을 이용한 암석의 일축압축강도 예측
직무분석을 위한 머신러닝 기반 전자문서 자동 분류 모델
140
머신러닝을 이용한 신경계통의 질환 퇴원환자의  중증도 보정 재원일수 예측 모형 개발
머신러닝 기법을 적용한 게임 산업 내 트위터 구전 효과 분석 : 구전의 양과 방향성 및 정보 불일치의 효과를 중심으로
API Call Time Interval을 활용한 머신러닝 기반의 악성코드 탐지
머신러닝을 활용한 TV 오디션 프로그램의 우승자 예측 모형 개발: 프로듀스X 101 프로그램을 중심으로
머신러닝을 위한 온톨로지 기반의 Raw Data 전처리 기법
불균형 데이터 환경에서 가계부채 상환연체 분류를 위한 머신러닝의 활용
머신 러닝을 이용한 트윗 데이터의 위치 추정 연구
머신러닝을 활용한 개인의 교통수단 선택 예측모형 구축
머신러닝 기법을 활용한 공장 에너지 사용량 데이터 분석
머신러닝 기법을 활용한 사교육 참여 예측 모형 탐색
150
머신러닝 알고리즘을 사용한 웨어러블 스마트 에어백에 관한 연구
머신 러닝을 이용한 경제분석
머신러닝 기법을 활용한 호텔산업 경기 동향 예측 연구
탠덤 용접에서의 용접선 추적용 아크길이 예측을 위한 머신러닝 알고리즘에 관한 연구
머신러닝을 위한 수학교육에 대한 연구
랜덤포레스트 머신러닝 알고리즘 기반 남·여 청소년의 자살생각 예측 및 분석
머신러닝 클러스터링을 이용한 컬럼 관측에 따른 대기 에어로

머신러닝 알고리즘이 적용된 가상화 내부 환경의 보안 인증벡터 생성에 대한 연구
TransMotion: 신체적 자기효능감 향상을 도와주는 머신러닝 기반의 발레체험 시스템에 관한 연구
머신러닝 기법을 통한 대한민국 부동산 가격 변동 예측
머신러닝기법을 이용한 상수리나무의 수간고별 직경추정 연구
식질머신: 만화 번역을 위한 심층 학습 기반 텍스트 제거 시스템
머신러닝 기반의 온실 제어를 위한 예측모델 개발
머신러닝 ELM과 VAR 모형을 이용한 오피스 임대료 예측에 관한 연구
인공지능 머신러닝을 이용한 체인 스프라켓 드라이브 시스템의 건전성 감시 장치
고장 확률 계산을 위한 서포트 벡터 머신 기반 몬테칼로 시뮬레이션 개선 방안
서포트 벡터 머신을 이용한 유사 소프트웨어 분류 모델의 설계
360
KPCA 및 서포트 벡터머신을 이용한 얼굴인식에 대한 연구
머신러닝을 이용한 공연문화예술 개인화 장르 추천 시스템
머신 러닝 기반 시각화를 통한 악성 댓글 문제 완화 연구
개인화 추천 시스템의 머신러닝을 위한 음수 미포함 행렬 분해와 변수 선택 기법의 비교
머신러닝을 활용한 인성교육 평가방안 연구
머신러닝과 Kano 모델을 활용한 팬 중심 축구선수 속성 모형 개발
머신러닝 기법을 활용한 아이돌 생존 가능성 예측 연구 : 산업 경쟁력 증진을 중심으로
머신러닝을 활용한 클라우드 인프라 비정상 모니터링
머신러닝 기반의 자동 정책 생성 방화벽 시스템 개발
머신 데이터 분석용 플랫폼 스플렁크를 이용한 취업지원 서비스 개선에 관한 연구 : 월드잡플러스 사례를 중심으로
370
머신러닝 기술을 활용한 건축물 사례기반 온실가스 감축 의사결정 지원 모델 구축
머신러닝을 이용한 검색량 기반 악성 URL 탐지기법 연구
머신러닝을 이용한 웹 공격 탐지 방안에 관한 연구
머신러닝을 활용한 서울시 아파트 물리적 특성 변수들의 비선형 영향 분석: 다변량 적응 회귀 스플라인 모형의 적용
의사결정나무와 서포트 벡터 머신 모델을 활용한 토지이용 변화 시뮬레이션: 통일 후 북한 도시를 대상으로
머신 러닝을 이용

한국어 텍스트 분석과 적용
기계학습 기반 다중셀 네트워크 역방향 성능 근사화
통계적 기계학습에서의 ADMM 알고리즘의 활용
커널기계 기법을 이용한 일반화 이분산자기회귀모형 추정
수문모형과 기계학습을 연계한 실시간 하천홍수 예측
추천키워드 및 기계학습을 이용한 문서 분류
안드로이드 악성코드 분석 및 기계학습 기법을 이용한 탐지 방법
RCM 자료와 기계학습을 이용한 북극권 카라-바렌츠 해역의 해빙면적비 예측
기계학습기법에 기반한 국제 유가 예측 모델
미세먼지 예측을 위한 기계학습 모델 간 성능 비교 연구:국내 발생 데이터를 중심으로
590
과학 기술 문헌 분석을 위한 기계학습 기반 범용 전문용어 인식 시스템
최근 건축분야의 인공지능 기계학습 연구동향 - 국내ㆍ외 연구논문을 중심으로 -
기계학습과 동적델타헤징을 이용한 옵션 헤지 전략
기계학습을 활용한 상품자산 투자모델에 관한 연구
기계학습 기법을 이용한 CNC 공구 마모도 예측에 관한 연구
Software Defined Networking을 위한 다중 기계학습 결합 기반의 DDoS 탐지 시스템
기계학습기반의 근사모델을 이용한 선박 횡동요 운동 예측
위키백과로부터 기계학습 기반 한국어 지식베이스 구축
Group Mnet 기계학습 기법을 활용한 고등학생의 진로정보 필요도 예측 변수 탐색
영작문 자동채점 시스템 개발에서 학습데이터 부족 문제 해결을 위한 앙상블 기법 적용의 효과
600
비만 폐쇄수면무호흡 환자에서 기계학습을 통한 적정양압 예측모형
트위터를 이용한 기계학습 기반의 영화흥행 예측
기계학습을 이용한 필터의 성능 저하 예측
컴퓨팅 자원의 가용성을 보장하기 위한 기계 학습 기반의 실시간 장애 예측 프레임워크 연구
기계학습 기반 저 복잡도 긴장 상태 분류 모델
연구논문 : 최소거리법과 기계학습법에 의한 한국어 텍스트의 저자 판별
기계학습 알고리즘의 컴퓨팅시간 단축을 위한 새로운 통계적 샘플링 기법
기계학습을 이용한 복숭아 경락가격 및 거래량 예측모형 비교
기계학습법을 이용한 서리 발생 구분 추정 연구
기계학습을 이

축사에서 딥러닝을 이용한 질병개체 파악방안
과학교과서 말뭉치(K-STeC) 학습을 통한 워드임베딩(word embedding) 모델의 정성적 성능 평가
CNN의 깊은 특징과 전이학습을 사용한 보행자 분류
인공지능 : 강화학습을 이용한 무인 자율주행 차량의 지역경로 생성 기법
딥 러닝을 이용한 한국어 형태소의 원형 복원 오류 수정
발전플랜트 성능데이터 학습에 의한 발전기 출력 추정 모델
무인항공기 영상 및 딥러닝 기반 객체인식 알고리즘을 활용한 해안표착 폐기물 탐지 기법 연구
불확실성 기반 상태 공간 학습 알고리즘을 이용한 Exploration-Exploitation 딜레마에 관한 연구
데이터 기반 확률론적 최적제어와 근사적 추론 기반 강화 학습 방법론에 관한 고찰
영어 스토리 북 읽기를 통한 구어영어 학습 가능성 - 인공지능번역기 활용 과업을 중심으로 - 이 논문은 2018년 김천대학교 교내학술연구비지원에 의한 것임(gc18068)
830
결함 검출을 위한 임베디드 딥러닝 시스템
환경적 요소 선별을 통한 딥러닝 기반 우울증 분류 예측
불균형 데이터 처리를 위한 과표본화 기반 앙상블 학습 기법
빅데이터로부터 추출된 주변 환경 컨텍스트를 반영한 딥러닝 기반 거리 안전도 점수 예측 모델
딥 러닝 알고리즘을 활용한 뇌파 분석 기반 졸음운전 사고예방 시스템
생활 패턴 인지를 위한 이벤트 연산 기반 예측 모델 학습 기법
자율주행 차량의 학습 데이터 자동 생성 시스템 개발
모바일 비전 및 위치 데이터 학습에 의한 장소 인식
Spiking Neural Networks(SNN) 구조에서 뉴런의 개수와 학습량에 따른 학습 성능 변화 분석
딥러닝 모델을 사용한 물체-사람 상호작용 패턴 출력 시스템
840
프로그래밍 학습을 위한 교육용 로봇 설계 및 구현
세종 전자사전과 준지도식 학습 방법을 이용한 용언의 어의 중의성 해소
무선 센서 네트워크에서 준지도 학습을 이용한 신경망 기반 노드 위치 추정
딥러닝을 이용한 화합물-단백질 상호작용 예측
베이지안 네트워크 학습을 이용한 방공 무기

...

교통정보 추론을 위한 비정형데이터 분석과 다중패턴저장 기법
VCM과 Beat Tracking을 이용한 음악의 명암 분류 기법 개발
확률기하와 기계학습을 통한 셀룰러 네트워크 순방향 성능 근사화
선택적 자질 차원 축소를 이용한 최적의 지도적 LSA 방법
컴퓨터 게임에서 사용자의 감성에 따른 게임 변화 기술
앙상블 인공신경망 기반의 암밴드형 지화 인식 시스템
악성코드 패킹유형 자동분류 기술 연구
랜덤 포레스트를 활용한 작품 가격 예측 모형 연구
Comparison and Analysis of P2P Botnet Detection Schemes
선형 SVM을 사용한 안드로이드 기반의 악성코드 탐지 및 성능 향상을 위한 Feature 선정
1520
시맨틱 텐서공간모델 기반 텍스트데이터 증식기법
한글 마이크로블로그 텍스트의 감정 분류 및 분석
이산 월시 변환이 메타모델을 사용한 유전 알고리즘에 미치는 영향
대면적 서셉터의 온도 균일도 검증 알고리즘
기계 장치와의 상호작용을 위한 실시간 저비용 손동작 제어 시스템
잠재디리슐레할당 기반 군집화를 통한 유사 범죄코드 발굴과 범죄예측
빌보드 스윕 스테레오 시차정합 알고리즘을 이용한 차량 검출 및 추적
클러스터링 기법을 이용한 암호화 화폐의 시각화 패턴 분석
인공신경망을 활용한 선형재료절단문제의 휴리스틱 기법 선택
계층형 시간적 메모리 네트워크를 기반으로 한 스트림 데이터의 연속 다중 예측
1530
번역 교육의 새로운 ‘통합' 패러다임 제안
딥뉴럴네트워크 기반의 네트워크 침입탐지시스템 설계
복수의 엣지 디바이스에서의 CNN 모델 분산 처리를 위한 축소된 분류 모델 활용 기법
대용량 자료에 대한 서포트 벡터 회귀에서 모수조절
SVM을 위한 교사 랭크 정규화
사용자 얼굴인식을 통한 자동출석체크 시스템
SDN에서 강화학습을 통한 멀티캐스트 트리 생성 기법 분석
소고기 육질 등급 예측을 위한 분류 알고리즘의 성능 연구
포트폴리오 최적화와 주가예측을 이용한 투자 모형
특허빅데이터 통계분석을 이용한 기술분석
1540
다양한 분류기법을 이용

생체신호처리를 이용한 비접촉식 열차 운전자 모니터링 시스템 개발
인공지능 로봇과 인간의 관계에 대한 윤리적 성찰과 전망
2D-QSAR방법을 이용한 농약류의 무지개 송어 급성 어독성 분석 및 예측
인가전압의 특성을 고려한 주거용 부하의 전류성분 추정기법 개발
IT행정에서 자동화행정행위에 관한 일반적 고찰
위키피디아 앵커 정보를 활용한 개체명 인식 오류 보정
M&W 파동 패턴과 유전자 알고리즘을 이용한 주식 매매 시스템 개발
ELM을 이용한 주거용 부하의 부하모델링 기법 개발
다중 클래스 SVM과 트리 분류를 이용한 제스처 인식 방법
ELM을 이용한 일별 최대 전력 수요 예측 알고리즘 개발
1740
Daphnia Magna Toxicity Bioassays Using Shadow Tracking Sensor & Patten Recognition
내부 온도를 이용한 PV 시스템 직류 접속반 진단용 인공 신경망 기반 예측기 설계
도로구간 유형별 차량 OBD2 데이터의 군집처리를 이용한 안전운전 등급 분류
빅데이터 통합모형 비교분석
특징에 따른 기어박스 결함의 진동신호 분석
전이에 의해 융합되는 시조의 문학치료 코드 연구
진로인식 향상을 위한 만5세 유아 진로코칭 프로그램 구안
언어 정보의 계량화와 시각화: 감성 어휘의 분포 의미 표현
불완전한 데이터를 처리할수 있는 분류기
인공지능과 법체계–전자인격론의 모순과 정보권한과의 갈등을 중심으로–
1750
의사결정트리를 이용한 돈사 환경데이터와 일당증체 간의 연관성 분석 모델 개발
SVM을 이용한 중계 로그 AP 탐지 기법
인간과 인공지능 로봇 캐릭터의 비교 연구-너도 인간이니? 기반으로
온라인 동향 분석을 위한 이벤트 문장 추출 방안
웹서버 로그 데이터의 이상상태 탐지 기법
비음수가중치제한과 네트워크 희소성을 고려한 오토인코더와 그래프오토인코더로 얻은 뇌 서브네트워크 비교
기어박스 결함 유형에 따른 고장진단을 위한 특징 분석
정렬불량 진단을 위한 유전알고리듬 기반 특징분석
다중 이미지에서 단일 이미지 검출 및 추적 시스템 

폐 결절 검출을 위한 합성곱 신경망의 성능 개선
이질 공공빅데이터 기반 농장별 맞춤 병해충 예측 모델
비 계층적 클러스터링 알고리즘 기반 저조도 상태 가시광 통신 시스템
AI 등을 활용한 사업자간 담합과 경쟁법의 대응
다중 사용자를 위한 Dynamic Time Warping 기반의 특징 강조형 제스처 인식 모델
TBCNN 기반 코드 클론 유형 분류
스포츠 기사 댓글에서 수사의문문으로 표현된 풍자 표현의 자동인식
사용자의 음악 선호요인 분석을 통한 개인 맞춤형 음악추천모델 연구
안개제거의 깊이 맵 추정을 위한 비선형 모델
소프트웨어 정의 무선 메쉬 네트워크에서 마스터 컨트롤러 선택방법
1960
XGBoost 모델 해석을 통한 노인의 인지능력 개선·악화 요인 탐
GPS의 위치 정보 시스템을 활용한 자율주행 차량의 차선 검출 구현
의료 인공지능에 대한 대한민국 영상의학과 전공의의 인식 조사 연구
수화번역시스템을 위한 인공신경망의 응용
조정절차에서 인공지능(AI)을 활용하는 방안
시간 간격 특징 벡터를 이용한 AdaBoost 기반 제스처 인식
대퇴의족의 자동 보행 모드 변경을 위한 랜덤 포레스트 기반 추정 모델 개발에 관한 연구
수정된 MobileNet을 이용한 과일의 결점 분류 알고리즘
단층 코어넷 다단입력 인공신경망회로의 함수에 관한 구현가능 연구
보편적 빅데이터와 빅데이터 교육의 방향성 연구 - 빅데이터 전문가의 인식 조사를 기반으로
1970
Gradle 빌드 오류 해결을 위한 솔루션 추천 방안
실시간 물체 검출을 위한 고효율 Viola-Jones 검출 프레임워크
외국인 투자자의 비정상적 중·장기매도성향패턴예측을 위한 지능형 조기경보시스템 구축
KNIME 분석 플랫폼 기반 스마트 미터 빅 데이터 클러스터링
자율주행 시대, 차세대 소프트웨어에서 길을 찾다
2차 통계량과 RNN을 이용한 OFDM 신호 검출
스마트공장을 위한 빅데이터 애널리틱스 플랫폼 아키텍쳐 개발
VCM과 Beat Tracking을 이용한 음악의 명암 분류 기법 개발
오픈소스 기반 빅데이터 플랫폼을

코로나 19와 부동산 감정평가
초분광 표적 탐지를 위한 L2,1-norm Regression 기반 밴드 선택 기법
Accurate and Efficient Log Template Discovery Technique
NB 모델을 이용한 형태소 복원
Crowdfunding Scams: The Profiles and Language of Deceivers
가중선형회귀를 통한 순항항공기의 궤적예측
토지 보상비 결정 요인 분석 - 건설CALS 데이터 중심으로
텍스트 마이닝을 이용한 감정 유발 요인‘Emotion Trigger'에 관한 연구
뇌출혈 동물 모델에서 부항이 뇌 인지기능 회복에 미치는 효과
와전류탐상검사를 이용하여 탐지 가능한 볼트홀 내부 균열 길이 연구
2170
자동 감성 인식을 위한 비교사-교사 분류기의 복합 설계
차량 번호판 인식을 위한 앙상블 학습기 기반의 최적 특징 선택 방법
해상 이미지를 활용한 3D 합성곱 신경망과 합성곱 장단기 메모리 신경망 기반의 유의 파고 추정
변분 오토인코더와 세그넷 기반 웨이퍼 불량패턴 탐지
Assessment of Premature Ventricular Contraction Arrhythmia by K-means Clustering Algorithm
크라우드펀딩에서의 사기 프로젝트 탐지
군집화 기반의 관계형 데이터 시각화를 위한 시각적 문법
전파 네트워크 특성 추출을 통한 소셜 미디어 속 루머 분류
미시추 구간의 지반 층상정보 예측을 위한 정규 크리깅 및 인공신경망 기법의 비교
그래프 데이터베이스 환경에서 이상징후 탐지를 위한 연관 관계 분석 기법
2180
문장 위치를 고려한 고객 리뷰 감성 분석 모형
다층신경망을 이용한 장래 생활폐기물 발생량 예측 연구 : B시 사례연구
자동외관검사를 위한 확률기반 불량 확인 알고리즘 개발
통시 말뭉치에 기반한 언어 변화 연구 -20세기 신문 말뭉치의 구축과 분석-
AI 어시스턴트의 요청문 인식 및 분류 주석을 위한 음악청취 관련 문형 패턴 연구
Fast Algorithm for Int

SWAT 및 random forest를 이용한 기후변화에 따른 한강유역의 수생태계 건강성 지수 영향 평가
결함 심각도에 기반한 소프트웨어 품질 예측
의료 문헌에서의 절차적 지식 추출
통계적 텍스트 마이닝을 이용한 빅 데이터 전처리
EXCUTE REAL-TIME PROCESSING IN RTOS ON 8BIT MCU WITH TEMP AND HUMIDITY SENSOR
한국어 의미역 결정을 위한 Korean PropBank 확장 및 도메인 적응 기술 적용
산업현장에서 작업자 안전을 위한 지능형 영상분석 시스템
단일 리드 심전도를 이용한 개인 식별
속성값 기반의 정규화된 로지스틱 회귀분석 모델
EEG 기반 감정인식을 위한 주석 레이블링과 EEG Topography 레이블링 기법의 비교 고찰
2370
연속 은닉 마르코프 모델을 이용한 GMA 용접품질 실시간 모니터링에 관한 연구
사회적 가치 측정방법의 특징 분석 및 최신 동향
SW교육에서 중학생의 핵심 역량 분석
Word2Vec과 WordNet 기반 불확실성 단어 간의 네트워크 분석에 관한 연구
통계적 특징 기반 인공신경망을 이용한 온라인 서명인식
인공지능 왓슨 기술과 보건의료의 적용
융합 인공벌군집 데이터 클러스터링 방법
클래스가 부가된 커널 주성분분석을 이용한 비선형 특징추출
사장교 케이블의 장력 추정을 위한 인공신경망 모델 개발
인공지능의 개인정보 자동화 처리가 야기하는 차별 문제에 관한 연구
2380
Quantified Lockscreen: 감정 마이닝과 자기정량화를 위한 개인화된 표정인식 및 모바일 잠금화면 통합 어플리케이션
대기경계층 고도 산출을 위한 인공신경망기법 적용
오픈 도메인 질의응답을 위한 검색문서 제약 및 정답유형 분류기술
T-EBOW를 이용한 취업알선 챗봇용 단문 분류 연구
영화 속 인공지능의 역할 변화에 대한 연구
자동분류기반 성격 유형별 도서추천시스템 개발을 위한 실험적 연구
자동화된 사실 확인(fact checking) 기술(technology)의 현황과 한계
API 특성 정보기반 악성 

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=979a972012fc6b8247de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0280754f15f72bd0b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0006f1a96dab3513b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7199cd63ed249a1dc85d2949c297615a',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=75236e331fc694674884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=bd0060b47bcefbf247de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=706f347e2422fe8fffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=168be367dfb0f97c4884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0b22ab4ffd2173dbc85d2949c297615a',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

2487

In [7]:
paper_url_1 = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url_1.append(tmp_url)

100%|█████████████████████████████████████████████████████████████████████████| 2487/2487 [00:00<00:00, 2469515.64it/s]


In [8]:
paper_url_1

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=979a972012fc6b8247de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0280754f15f72bd0b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0006f1a96dab3513b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7199cd63ed249a1dc85d2949c297615a',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=75236e331fc694674884a65323211ff0',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=bd0060b47bcefbf247de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=706f347e2422fe8fffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=168be367dfb0f97c4884a65323211ff0',
 'http://www.ris

In [9]:
len(paper_url_1)

2487

# 검색키워드: MachineLearning

In [10]:
#http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=MachineLearning&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2014%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=MachineLearning

In [11]:
paper_link_list = []
#2
for i in range(2):
    before_url = "http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=MachineLearning&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2014%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=MachineLearning"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

머신러닝을 활용한 사상체질 분류 모델 선정과 서비스 플로우 디자인
Terra MODIS NDVI 및 LST 자료와 RNN-LSTM을 활용한 토양수분 산정
머신러닝을 이용한 알루미늄 전해 커패시터 고장예지
MIMO-OFDM 시스템에서 에너지 효율성을 위한 기계 학습 기반 적응형 전송 기술 및 Feature Space 연구
소설 코퍼스를 활용한 ‘서동산'의 소설 『염마』의 저자 판별
API Call Time Interval을 활용한 머신러닝 기반의 악성코드 탐지
비과 영역에서 머신러닝의 적용
금융 데이터 및 텍스트 데이터를 활용한 금융 기업 조기 경보 모형 개발 : 부실은행 예측을 중심으로
기계학습을 이용한 역사 텍스트의 저자판별: 1920년대 개벽 잡지의 논설 텍스트
기계 학습을 활용한 마그네슘 합금의 통전 비선형 온도 예측
10
머신러닝 적용 과일 수확시기 예측시스템 설계 및 구현
신경망을 이용한 소프트웨어 취약 여부 예측 시스템
뉴럴 디코딩의 원리와 최신 연구 동향 소개
13


In [12]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=be91c45bcfc9c76f47de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b53e90b4935390537f7a54760bb41745',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=31918cdfbc0bec6d6aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fe5f583b4d24cef1d18150b21a227875',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f0c46111c930cc2b7f7a54760bb41745',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6a7687175a5b82876aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=af7437d1a05001324884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=86bc6d3b54fcc80f47de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=309f6b7c18cd61e74884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [13]:
len(paper_link_list)

13

In [14]:
paper_url_2 = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url_2.append(tmp_url)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<?, ?it/s]


In [15]:
paper_url_2

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=be91c45bcfc9c76f47de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b53e90b4935390537f7a54760bb41745',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=31918cdfbc0bec6d6aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fe5f583b4d24cef1d18150b21a227875',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f0c46111c930cc2b7f7a54760bb41745',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6a7687175a5b82876aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=af7437d1a05001324884a65323211ff0',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=86bc6d3b54fcc80f47de9c1710b0298d',
 'http://www.ris

In [16]:
len(paper_url_2)

13

# 검색키워드: 머신 러닝(머신러닝)

In [17]:
#http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D

In [18]:
paper_link_list = []
#69
for i in range(69):
    before_url = "http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

머신러닝 인공지능과 인간전문직의 협업의 의미와 법적 쟁점
머신러닝 인공지능의 법 분야 적용의 현재와 미래
머신러닝 기법을 활용한 사교육 참여 예측 모형 탐색
머신러닝 알고리즘을 이용한 MBS 조기상환율 예측
머신 러닝을 활용한 회사 SNS 메시지에 내포된 심리적 거리 추출 연구
머신러닝 기법과 TBM 시공정보를 활용한 토압식 쉴드TBM 굴진율 예측 연구
자동화 머신러닝 시스템의 개념적 디자인을 적용한 프로토타입의 개발
간접차별과 머신러닝에 의한 특성추론
머신러닝 기술의 광업 분야 도입을 위한 활용사례 분석
머신러닝에 관한 OSS 라이선스 연구
10
머신러닝 기법 기반의 예측조합 방법을 활용한 산업 부가가치율 예측 연구
행정-정책 의사결정에서 머신러닝(machine learning) 방법론 도입의 정책적 함의: 기계의 한계와 증거기반 의사결정(evidence-based decision-making)
중학생의 진로 결정 예측변수 탐색 : 머신러닝 기법 적용
머신 러닝 기반 소셜 빅데이터 분석을 이용한 금융자산 트레이딩 모델의 성능 향상에 관한 연구
머신러닝을 위한 불균형 데이터 처리 방법 : 샘플링을 위주로
중학생의 진로 결정 예측변수 탐색 : 머신러닝 기법 적용
머신러닝 기법을 이용한 수도권 지역의 호우피해 예측함수 개발
머신러닝 적용 과일 수확시기 예측시스템 설계 및 구현
UX 디자인 과정에서의 머신러닝 활용 방법
인적자원의 부정채용을 감시하기 위한 머신러닝의 활용
20
고빈도 자료를 이용한 머신러닝모형의 예측력 비교 ․ 분석: KOSPI200 선물시장을 중심으로
데이터 이산화와 계층형 클러스터링 알고리즘을 적용한 효과적인 머신 러닝 방법 관한 연구
머신러닝 플랫폼을 활용한 소프트웨어 교수-학습 모형 개발
머신 러닝을 활용한 과학 논변 구성 요소 코딩 자동화 가능성 탐색 연구
머신러닝 모델링을 위한 웹 기반 위성영상 데이터 전처리 자동화 시스템 설계 및 구현
머신러닝 ELM과 VAR 모형을 이용한 오피스 임대료 예측에 관한 연구
사용자 성향 그룹 분류를 위한 

낙하 충격에 의한 탄소-케블라 하이브리드 직물의 전압 강하 특성과 머신 러닝을 통한 충격체 형상 예측
소셜미디어 콘텐츠 주제와 고객 인게이지먼트 간의 관계분석: 머신러닝 방법론을 중심으로
인공지능(AI) 스피커의 사회적 가치 실현 가능성 예측 머신러닝 알고리즘의 적용
영화 흥행에 영향을 미치는 새로운 변수 개발과이를 이용한 머신러닝 기반의 주간 박스오피스 예측
머신 러닝 회귀 방안을 이용한 인공지지체 기공 크기 예측모델 성능에 관한 연구
머신러닝 기반의 자동 정책 생성 방화벽 시스템 개발
머신러닝 알고리즘 분석 및 비교를 통한 Big-5 기반 성격 분석 연구
머신러닝 기법을 활용한 대용량 시계열 데이터 이상 시점탐지 방법론 : 발전기 부품신호 사례 중심
LCD 검사 공정에서 가상 계측을 위한 머신 러닝 기반 예측 모델
머신러닝 기반의 디지털 방송 프로그램 유형 분류 및 활용 방안 연구
240
한의 체중 조절 프로그램에 참여한 과체중, 비만 환자에서의 머신러닝 기법을 적용한 체중 감량 예측 연구
도심지 자율주행을 위한 머신러닝 기반의 실시간 다 객체 인식 방법
트랜잭션 기반 머신러닝에서 특성 추출 자동화를 위한 딥러닝 응용
특징 선택을 활용한 머신 러닝 기반의 위암 컴퓨터 보조 진단 시스템
머신러닝 기반 CFS(Correlation-based Feature Selection)기법과 Random Forest모델을 활용한 BMI(Benthic Macroinvertebrate Index) 예측에 관한 연구
머신러닝 기반 안드로이드 모바일 악성 앱의최적 특징점 선정 및 모델링 방안 제안
연구논문 : 모돈도태 의사결정지원을 위한 머신러닝모델 응용
머신 러닝을 이용한 중유 화력 열병합 발전소 보일러의 고장 예지
머신 러닝을 이용한 영상 특징 기반 전기차 검출 및 분류 시스템
머신러닝 기반 건강컨설팅 성공여부 예측모형 개발
250
머신 러닝을 이용한 외란 관측기 구현
머신러닝을 활용한 심근경색증/협심증 예측 및 주요 위험요인 선별
광역시 지역주민의 원전계속운전 수용성 결정요

선수 스마트 에이전트의 법적 지위에 관한 연구
사이버 공격에 의한 시스템 이상상태 탐지 기법
한국 사회복지학의 최근 연구경향: 연관규칙 분석의 활용
자연어 처리 기반 맞춤형 트윗 추천 시스템
폴립 가중치 영상 생성을 통한 캡슐내시경 영상의학습 성능 비교 연구
가상 환경에서의 강화학습을 이용한 비행궤적 시뮬레이션
이미지 생성을 위해 노이즈를 이용한 GAN 시스템
XGBoost와 Word2Vec을 이용한 온라인 쇼핑 패턴 기반 하이브리드 협업 필터링
Filter Method와 Classification 알고리즘을 이용한 전자상거래 블랙컨슈머 탐지에 대한 연구
이미지 인식과 캡션을 위한 기계학습 모델 연구
450
ML 기반의 전기추진시스템을 위한 PCS에 관한 연구
감시 영상을 활용한 OpenPose 기반 아동 학대 판단시스템
에너지 인터넷을 위한 GRU기반 전력사용량 예측
수치 데이터 세트에서 Tomek Links 방법과 Balancing GAN을 결합한 불균형 데이터 문제 개선 기술
딥러닝 모형의 복잡도에 관한 연구
다중 웹 데이터와 LSTM을 사용한 전염병 예측
딥러닝 기반 초·중등 학습용 나무 인식 기법
동계 전력수요예측을 위한 신경망 모델에 관한 연구
파이썬 코딩을 도입한 수학 교과 지도 방안 개발 - 2015 개정 교육과정 중학교 수학 교과의 ‘소인수분해' 내용을 중심으로 -
유동해석을 통한 스마트 제어밸브 학습 및 성능평가용 프로파일 모델링
460
공공 빅데이터를 활용한 안 질환에 영향을 미치는 기상 요인 분석
크라우드센싱 시스템에서 머신러닝을이용한 이상데이터 탐지
대용량 자료에 대한 서포트 벡터 회귀에서 모수조절
별색의 색 예측을 위한 기본 잉크 배합 비율을 결정하는 인공지능 알고리즘에 대한 연구 - 패키징 인쇄시장에서 기본 잉크 배합을 통한 별색 예측을 오프셋 인쇄 중심으로
빅데이터 구축을 위한 알루미나 테이프 캐스팅 공정 최적화
인공지능 피아노 교습 기술의 분석과 전망
의료 인공지능에 대한 형법적 고찰 -왓슨(Watson)을 중심으로-
희소한 텍스

서울시 노인종합복지시설의 서비스 부문별 공간 스마트화에 관한 연구
ROC를 이용한 보행에 영향을 미치는 한계강우량의 정확도 평가
광업 분야의 딥러닝 기술 적용연구 사례분석
사회적 가치 측정방법의 특징 분석 및 최신 동향
RBFNNs 기반 소프트맥스 패턴분류기를 이용한 포즈변화에 강인한 얼굴인식 시스템 설계
국내외 초·중등학교 인공지능 교육과정 분석
漢文科에서의 傳統的 價値觀 授業 方案 硏究 - Google Docs 활용을 기반으로 -
K-Means 클러스터링을 적용한 향상된 CS-RANSAC 알고리즘
4차 산업혁명 시대 문학의 현재와 미래
미국 모기지 대출의 인종차별성에 대한 랜덤 포리스트 분석
670
인공지능(AI)의 창작물에 대한 지식재산권 보호방안
실내외 환경과 사용자의 행동을 고려한 스마트 홈 서비스 시스템
인간과 인공지능 로봇 캐릭터의 비교 연구-너도 인간이니? 기반으로
인공지능(Artificial Intelligence)과 대학수학교육
일상생활 신체활동 측정을 위한 스마트폰기반 검사도구의수렴타당도검증
12주간 유산소 운동이 중년여성의 대사증후군 위험인자와 C-반응성단백질 및 호모시스테인에 미치는 영향
공무원 이직의도의 연관요인에 대한 추론과 예측에 관한 연구
딥러닝 알고리즘을 활용한 천식 환자 발생 예측에 대한 연구
원전기기의 상태 감시 및 진단 기술
Post–COVID-19 시대의 새로운 정신건강서비스: 자폐범주성장애에의 적용 현황
680
“사법시스템과 사법환경에서의 인공지능 이용에 관한 유럽 윤리헌장”의 검토 - 민사사법절차에서의 인공지능 도입 논의와 관련하여 -
인공지능의 개인정보 자동화 처리가 야기하는 차별 문제에 관한 연구
“사법시스템과 사법환경에서의 인공지능 이용에 관한 유럽 윤리헌장”의 검토 - 민사사법절차에서의 인공지능 도입 논의와 관련하여 -
고속도로 이동경로별 중장기 통행시간 예측 방법론 개발
세법상 법률 인공지능의 한계와 그 영향에 관한 試論的 考察
수질-수리 인자를 고려한 상수도 관망 사이버 공격 탐지 모델 개발
인공지능 의사결

In [19]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=48be253dd0bf0f72b7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=195fe765b820bb10b7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=684f9c0f7627a9c77ecd42904f0c5d65',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6bea7b1de99b4d03b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=367910d5497b3bbc7ecd42904f0c5d65',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2575b7ba56f2aa92e9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=616cb326e8b040dab36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=289ee9a34b41a9714884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ccc7a130edfc66f2b36097776a77e665',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [20]:
len(paper_link_list)

690

In [21]:
paper_url_3 = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url_3.append(tmp_url)

100%|████████████████████████████████████████████████████████████████████████████████████████| 690/690 [00:00<?, ?it/s]


In [22]:
paper_url_3

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=48be253dd0bf0f72b7998d826d417196',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=195fe765b820bb10b7998d826d417196',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=684f9c0f7627a9c77ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=6bea7b1de99b4d03b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=367910d5497b3bbc7ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2575b7ba56f2aa92e9810257f7042666',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=616cb326e8b040dab36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=289ee9a34b41a9714884a65323211ff0',
 'http://www.ris

In [23]:
len(paper_url_3)

690

In [24]:
paper_url = paper_url_1 + paper_url_2 + paper_url_3
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=979a972012fc6b8247de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0280754f15f72bd0b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=0006f1a96dab3513b36097776a77e665',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=7199cd63ed249a1dc85d2949c297615a',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=75236e331fc694674884a65323211ff0',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=bd0060b47bcefbf247de9c1710b0298d',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=706f347e2422fe8fffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=168be367dfb0f97c4884a65323211ff0',
 'http://www.ris

In [25]:
len(paper_url)

3190

In [26]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [27]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [28]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [29]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./engin_이산화탄소.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 ERROR
10 paper clear
11 paper clear
12 ERROR
13 paper clear
14 paper clear
15 ERROR
16 paper clear
17 ERROR
18 ERROR
19 paper clear
20 paper clear
21 paper clear
22 ERROR
23 paper clear
24 paper clear
25 ERROR
26 paper clear
27 paper clear
28 paper clear
29 ERROR
30 paper clear
31 paper clear
32 ERROR
33 paper clear
34 paper clear
35 ERROR
36 paper clear
37 paper clear
38 paper clear
39 ERROR
40 paper clear
41 ERROR
42 paper clear
43 ERROR
44 paper clear
45 ERROR
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 ERROR
62 paper clear
63 paper clear
64 paper clear
65 ERROR
66 paper clear
67 paper clear
68 paper clear
69 paper clear
70 ERROR
71 paper clear
72 paper clear
73 paper clear
74

540 paper clear
541 paper clear
542 ERROR
543 paper clear
544 paper clear
545 paper clear
546 paper clear
547 paper clear
548 ERROR
549 paper clear
550 ERROR
551 paper clear
552 paper clear
553 paper clear
554 paper clear
555 paper clear
556 ERROR
557 paper clear
558 paper clear
559 paper clear
560 paper clear
561 paper clear
562 paper clear
563 paper clear
564 paper clear
565 ERROR
566 paper clear
567 paper clear
568 paper clear
569 paper clear
570 paper clear
571 paper clear
572 paper clear
573 paper clear
574 paper clear
575 ERROR
576 paper clear
577 paper clear
578 paper clear
579 paper clear
580 paper clear
581 paper clear
582 paper clear
583 paper clear
584 paper clear
585 paper clear
586 paper clear
587 paper clear
588 paper clear
589 paper clear
590 paper clear
591 paper clear
592 paper clear
593 ERROR
594 paper clear
595 paper clear
596 paper clear
597 paper clear
598 paper clear
599 paper clear
600 paper clear
601 paper clear
602 paper clear
603 paper clear
604 paper clear
60

1061 paper clear
1062 paper clear
1063 paper clear
1064 paper clear
1065 paper clear
1066 paper clear
1067 paper clear
1068 paper clear
1069 paper clear
1070 paper clear
1071 paper clear
1072 paper clear
1073 paper clear
1074 paper clear
1075 paper clear
1076 paper clear
1077 paper clear
1078 paper clear
1079 paper clear
1080 paper clear
1081 paper clear
1082 ERROR
1083 ERROR
1084 ERROR
1085 paper clear
1086 paper clear
1087 paper clear
1088 paper clear
1089 paper clear
1090 paper clear
1091 paper clear
1092 ERROR
1093 paper clear
1094 paper clear
1095 paper clear
1096 paper clear
1097 paper clear
1098 ERROR
1099 paper clear
1100 paper clear
1101 paper clear
1102 paper clear
1103 paper clear
1104 paper clear
1105 paper clear
1106 paper clear
1107 paper clear
1108 paper clear
1109 paper clear
1110 paper clear
1111 paper clear
1112 paper clear
1113 paper clear
1114 ERROR
1115 paper clear
1116 paper clear
1117 paper clear
1118 ERROR
1119 paper clear
1120 paper clear
1121 paper clear
1122 

1556 ERROR
1557 paper clear
1558 paper clear
1559 paper clear
1560 paper clear
1561 paper clear
1562 paper clear
1563 paper clear
1564 paper clear
1565 paper clear
1566 paper clear
1567 paper clear
1568 paper clear
1569 paper clear
1570 paper clear
1571 paper clear
1572 paper clear
1573 paper clear
1574 paper clear
1575 paper clear
1576 ERROR
1577 ERROR
1578 paper clear
1579 paper clear
1580 paper clear
1581 paper clear
1582 paper clear
1583 paper clear
1584 paper clear
1585 paper clear
1586 paper clear
1587 paper clear
1588 paper clear
1589 paper clear
1590 paper clear
1591 ERROR
1592 paper clear
1593 paper clear
1594 paper clear
1595 paper clear
1596 paper clear
1597 paper clear
1598 paper clear
1599 paper clear
1600 paper clear
1601 paper clear
1602 paper clear
1603 paper clear
1604 paper clear
1605 paper clear
1606 paper clear
1607 paper clear
1608 paper clear
1609 paper clear
1610 paper clear
1611 paper clear
1612 paper clear
1613 paper clear
1614 paper clear
1615 paper clear
1616

2049 paper clear
2050 paper clear
2051 paper clear
2052 paper clear
2053 paper clear
2054 paper clear
2055 paper clear
2056 paper clear
2057 paper clear
2058 paper clear
2059 paper clear
2060 paper clear
2061 paper clear
2062 paper clear
2063 paper clear
2064 ERROR
2065 paper clear
2066 paper clear
2067 paper clear
2068 paper clear
2069 paper clear
2070 paper clear
2071 paper clear
2072 paper clear
2073 paper clear
2074 paper clear
2075 paper clear
2076 paper clear
2077 ERROR
2078 paper clear
2079 paper clear
2080 paper clear
2081 paper clear
2082 paper clear
2083 paper clear
2084 paper clear
2085 paper clear
2086 ERROR
2087 paper clear
2088 paper clear
2089 paper clear
2090 paper clear
2091 paper clear
2092 paper clear
2093 paper clear
2094 paper clear
2095 paper clear
2096 paper clear
2097 paper clear
2098 paper clear
2099 paper clear
2100 paper clear
2101 paper clear
2102 paper clear
2103 paper clear
2104 ERROR
2105 paper clear
2106 paper clear
2107 paper clear
2108 paper clear
2109

2541 paper clear
2542 paper clear
2543 paper clear
2544 paper clear
2545 paper clear
2546 paper clear
2547 paper clear
2548 paper clear
2549 paper clear
2550 paper clear
2551 paper clear
2552 paper clear
2553 paper clear
2554 paper clear
2555 paper clear
2556 paper clear
2557 paper clear
2558 paper clear
2559 paper clear
2560 paper clear
2561 paper clear
2562 paper clear
2563 paper clear
2564 paper clear
2565 paper clear
2566 paper clear
2567 paper clear
2568 paper clear
2569 paper clear
2570 paper clear
2571 paper clear
2572 paper clear
2573 paper clear
2574 paper clear
2575 paper clear
2576 paper clear
2577 paper clear
2578 paper clear
2579 paper clear
2580 paper clear
2581 paper clear
2582 paper clear
2583 paper clear
2584 paper clear
2585 paper clear
2586 paper clear
2587 paper clear
2588 paper clear
2589 paper clear
2590 paper clear
2591 paper clear
2592 paper clear
2593 paper clear
2594 paper clear
2595 paper clear
2596 paper clear
2597 paper clear
2598 paper clear
2599 paper cle

3033 paper clear
3034 paper clear
3035 paper clear
3036 paper clear
3037 paper clear
3038 paper clear
3039 paper clear
3040 paper clear
3041 paper clear
3042 paper clear
3043 paper clear
3044 paper clear
3045 paper clear
3046 paper clear
3047 paper clear
3048 paper clear
3049 paper clear
3050 paper clear
3051 paper clear
3052 paper clear
3053 paper clear
3054 paper clear
3055 paper clear
3056 paper clear
3057 paper clear
3058 paper clear
3059 paper clear
3060 paper clear
3061 paper clear
3062 paper clear
3063 paper clear
3064 paper clear
3065 paper clear
3066 paper clear
3067 paper clear
3068 paper clear
3069 paper clear
3070 paper clear
3071 paper clear
3072 paper clear
3073 paper clear
3074 paper clear
3075 ERROR
3076 paper clear
3077 ERROR
3078 paper clear
3079 paper clear
3080 paper clear
3081 paper clear
3082 paper clear
3083 paper clear
3084 paper clear
3085 paper clear
3086 paper clear
3087 paper clear
3088 paper clear
3089 paper clear
3090 paper clear
3091 paper clear
3092 pape